In [1]:
conf_list = ["https://www.inf.fu-berlin.de/inst/ag-ti/socg22/socg.html"]

In [2]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from g4f.client import Client

client = Client()
def ask(promt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": promt}],
    )
    print(response.choices[0].message.content)

user_agent_mac = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Safari/605.1.15"
user_agent_linux = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:125.0) Gecko/20100101 Firefox/125.0" 
headers = {"User-Agent": user_agent_linux} 

In [3]:
def extract_information(url):
    request = Request(url, None, headers) 
    response = urlopen(request)
    html = response.read() 
    soup = BeautifulSoup(html, features="html.parser")

    for script in soup(["script", "style"]):
        script.extract()    

    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    promt = 'find and return 4 lists (in structure: (number of list) : [only data in list, without any artifacts]): first includes only: "topics", second  includes only: "speakers", third includes only: "time and data",  fourth includes only:"place" ' + text
    ask(promt)

In [4]:
url = "https://www.inf.fu-berlin.de/inst/ag-ti/socg22/socg.html"
extract_information(url)

New g4f version: 0.3.0.10 (current: 0.3.0.8) | pip install -U g4f
1 : ["Design, analysis, and implementation of geometric algorithms and data structures", "Computational complexity of geometric problems", "Implementation and experimental evaluation of geometric algorithms and heuristics, including mathematical, numerical, and algebraic aspects", "Discrete and combinatorial geometry", "Computational topology, topological data analysis, and topological combinatorics", "Applications of computational geometry in any field"]
2 : ["Henry Adams (Colorado State University, USA)", "SangWon Bae (Kyonggi University, South Korea)", "Édouard Bonnet (ENS Lyon, France)", "Timothy Chan (University of Illinois at Urbana-Champaign, USA)", "Hsien-Chih Chang (Darthmouth College, USA)", "Hu Ding (University of Science and Technology, China)", "Vida Dujmović (University of Ottawa, Canada)", "Herbert Edelsbrunner (IST, Austria)", "Radoslav Fulek (UC San Diego, USA)", "Xavier Goaoc; co-chair (Université de Lo